# SQD Algorithm Ground State Finder

## Project Overview

This project combines **Unitary Coupled-Cluster Jastrow (UCJ) ansatz** with the **Subspace Quantum Deflation (SQD) method** to find the ground state energy of molecules, specifically the N₂ (nitrogen) molecule.

**Key Features:**
- Uses quantum circuits to prepare initial trial states
- Employs the SQD method to recover ground state configurations
- Integrates with IBM Quantum cloud services for real quantum execution
- Supports ab-initio electronic structure calculations using PySCF
- Implements fermionic and qubit Hamiltonian transformations

**Technology Stack:**
- Qiskit (quantum circuit framework)
- OpenFermion (fermionic operator handling)
- Qiskit-addon-sqd (SQD algorithm implementation)
- PySCF (quantum chemistry calculations)
- ffsim (efficient fermionic circuit simulation)

## Project Architecture

The project is organized into modular components:

### 1. **Hamiltonians** (`src/hamiltonians/`)
Responsible for setting up the electronic structure problem:
- **`fermionic_hamiltonian.py`**: Builds the fermionic Hamiltonian from ab-initio calculations using PySCF
  - Generates molecular integrals for the N₂ molecule
  - Returns a fermionic operator representation
  - Supports different basis sets and active spaces

- **`qubit_hamiltonian.py`**: Transforms fermionic operators into qubit operators
  - Applies Jordan-Wigner transformation
  - Converts fermionic algebra to qubit operations

### 2. **Ansatz** (`src/ansatz/`)
Defines the quantum circuit structure:
- **`LUCJ_ansatz.py`**: Implements the LUCJ ansatz for quantum state preparation
  - Creates parameterized quantum circuits
  - Uses efficient fermionic circuit representations (ffsim)
  - Configurable number of layers/repetitions

- **`initial_parameters.py`**: Initializes circuit parameters
  - Uses CCSD (Coupled-Cluster Singles Doubles) amplitudes
  - Provides good starting point for optimization

- **`run_on_ibm.py`**: Submits quantum circuits to IBM Quantum backends
  - Handles authentication with IBM cloud
  - Selects least-busy backend
  - Collects measurement results

### 3. **Measurement** (`src/measure/`)
Processes quantum results for ground state recovery:
- **`subspace_hamiltonian.py`**: Implements the SQD method
  - Configuration recovery from bitstrings
  - Iterative refinement of ground state
  - Uses fermionic solver in subspace

## Workflow: How the Algorithm Works

### Step 1: Electronic Structure Setup
```
Molecule Specification (N₂)
    ↓
PySCF ab-initio calculation
    ↓
Fermionic Hamiltonian H_f
    ↓
Active Space Selection (10 electrons, 8 orbitals)
```

### Step 2: Quantum Circuit Preparation
```
Fermionic Hamiltonian H_f
    ↓
Jordan-Wigner Transformation
    ↓
Qubit Hamiltonian H_q
    ↓
Initialize CCSD parameters
    ↓
Build LUCJ Ansatz Circuit |ψ(θ)⟩
```

### Step 3: Quantum Measurement
```
Execute on IBM Quantum Backend
    ↓
Collect measurement bitstrings
    ↓
Extract probability distribution
    ↓
Save counts (bitstring frequency data)
```

### Step 4: Ground State Recovery (SQD)
```
Measurement results (bitstrings + probabilities)
    ↓
Configuration recovery
    ↓
Subspace diagonalization (multiple batches)
    ↓
Iterative refinement
    ↓
Ground state energy estimate E_0

## Installation & Setup

### 1. Install Dependencies
```bash
pip install -r requirements.txt
```

**Main packages:**
- `qiskit` - Quantum circuit framework
- `pyscf` - Quantum chemistry ab-initio solver
- `openfermion` - Fermionic operator algebra
- `qiskit-addon-sqd` - SQD algorithm
- `qiskit-ibm-runtime` - IBM Quantum cloud API

### 2. IBM Quantum Setup (Optional)
To run on real quantum hardware:
```bash
# Set environment variables
export QISKIT_IBM_TOKEN="your_api_token_here"
export QISKIT_IBM_INSTANCE="ibm-q/open/main"  # optional
```

Get your API token from: https://quantum.ibm.com/


## Basic Usage

### Approach 1: Build and Execute Circuit
```python
from src.ansatz.LUCJ_ansatz import build_ucj_circuit

# Step 1: Create the circuit
circuit, hamiltonian = build_ucj_circuit(
    bond_length=1.0977,  # N-N bond length in Angstrom
    basis="6-31g",       # Basis set
    n_reps=2,            # Number of ansatz layers
    active_space=(10, 8) # (electrons, orbitals)
)

# Step 2: Execute on IBM Quantum
from src.ansatz.run_on_ibm import run_ansatz_on_ibm

counts = run_ansatz_on_ibm(
    circuit=circuit,
    shots=1024,
    backend="ibmq_qasm_simulator"
)
```

### Approach 2: SQD Ground State Recovery
```python
from src.measure.subspace_hamiltonian import run_sqd_from_counts

# Use measurement results to recover ground state
energies, occupancies, history = run_sqd_from_counts(
    counts_file="ibm_counts.json",
    iterations=5,
    n_batches=5,
    active_space=(10, 8)
)
```

## Key Concepts

### LUCJ Ansatz (Unitary Coupled-Cluster Jastrow)
An **ansatz** is a parameterized quantum circuit that prepares a trial state. The LUCJ ansatz:
- Combines unitary coupled-cluster operators with Jastrow factors
- Efficiently encodes electron correlations
- Uses parameters initialized from classical CCSD theory
- Can represent ground states of molecular systems

### Subspace Quantum Deflation (SQD)
A post-processing method that:
- Takes bitstring frequencies from quantum measurements
- Recovers electronic configurations using fermionic operators
- Uses classical diagonalization in smaller subspaces
- Iteratively finds ground state without full wavefunction reconstruction
- Mitigates quantum noise through configuration recovery

### Active Space
Quantum chemistry approximation where:
- Only the most *important* electrons and orbitals are treated quantum mechanically
- For N₂: 10 electrons in 8 orbitals (out of many total)
- Core electrons frozen, kept at Hartree-Fock level
- Reduces circuit complexity while maintaining accuracy

### Bond Length Sweep
The algorithm can compute energies for different N₂ bond lengths:
- Find potential energy surface (PES)
- Optimize molecular geometry
- Study chemical bonding

In [ ]:
# Example 1: Build and visualize the LUCJ circuit
import sys
sys.path.insert(0, '/Users/hebron/Documents/SQD-Algorithm-Groundstatefinder')

from src.ansatz.LUCJ_ansatz import build_ucj_circuit

# Create circuit for N2 at equilibrium bond length
circuit, hamiltonian = build_ucj_circuit(
    bond_length=1.0977,  # Angstrom
    basis="6-31g",
    n_reps=2,  # 2 layers
    active_space=(10, 8),
    build_hamiltonian=True
)

print(f"Circuit has {circuit.num_qubits} qubits")
print(f"Circuit depth: {circuit.depth()}")
print(f"Number of parameters: {circuit.num_parameters}")
print(f"\nCircuit:\n{circuit}")

In [ ]:
# Example 2: Build Fermionic and Qubit Hamiltonians
from src.hamiltonians.fermionic_hamiltonian import build_n2_fermionic_operator
from src.hamiltonians.qubit_hamiltonian import build_qubit_hamiltonian

# Build fermionic Hamiltonian
print("Building fermionic Hamiltonian for N2...")
fermionic_h = build_n2_fermionic_operator(
    bond_length=1.0977,
    basis="6-31g",
    active_space=(10, 8)
)
print(f"Fermionic Hamiltonian built")
print(f"Number of fermionic terms: {len(fermionic_h.normal_ordered())}")

# Transform to qubit Hamiltonian (Jordan-Wigner)
print("\nTransforming to qubit Hamiltonian...")
qubit_h = build_qubit_hamiltonian(fermionic_h)
print(f"Qubit Hamiltonian built")
print(f"Number of qubit terms: {len(qubit_h)}")
print(f"\nSample qubit terms:")
terms = list(qubit_h.terms.items())[:3]
for pauli_str, coeff in terms:
    print(f"  {coeff:.4f} * {pauli_str}")

## Data Format: IBM Counts File

The `ibm_counts.json` file stores quantum measurement results:

```json
{
  "counts": {
    "00000000000000": 145,
    "00000000000001": 23,
    "00000000000010": 18,
    ...
  },
  "num_qubits": 14,
  "shots": 1024,
  "bond_length": 1.0977,
  "basis": "6-31g",
  "metadata": {...}
}
```

**Fields:**
- `counts`: Dictionary where keys are bitstrings and values are measurement counts
- `num_qubits`: Number of qubits in the circuit
- `shots`: Total number of measurements
- Optional metadata about the circuit parameters

This file is produced by `run_on_ibm.py` and consumed by `subspace_hamiltonian.py` for SQD analysis.


## Project Workflow Summary

```
Start
  ↓
1. Define N₂ molecule (bond length, basis set)
  ↓
2. Build fermionic Hamiltonian via PySCF
  ↓
3. Transform to qubit Hamiltonian (Jordan-Wigner)
  ↓
4. Prepare LUCJ ansatz circuit with CCSD parameters
  ↓
5. Execute on IBM Quantum backend (or simulator)
  ↓
6. Collect bitstring counts → save to JSON
  ↓
7. Run SQD configuration recovery
  ↓
8. Recover ground state energy and properties
  ↓
End
```

## Next Steps

- Explore individual modules in `src/` for detailed implementation
- Run `Example 1` and `Example 2` cells above to verify setup
- Check `ibm_counts.json` for sample measurement data
- Modify parameters (bond length, basis set, n_reps) to experiment